# Critic

In [1]:
from hw1 import Critic, MidiCriticDataProcessor

In [2]:
# Test code for critic
print("Testing critic ...")
processor = MidiCriticDataProcessor(data_directory='.')
train_loader, test_loader = processor.prepare_data()

Testing critic ...


#### Train new model

In [3]:
print("Loading critic data ...")
criticTr = Critic(load_trained=False)

Loading critic data ...


In [4]:
test_loader.dataset[0][0]

tensor([196., 259., 256., 197., 260., 256., 373.,  92., 370.,  80., 256., 366.,
         65., 220., 256., 208., 261., 256., 193., 256., 363.,  67., 259., 256.,
        195., 363.,  69., 260., 256., 197., 261., 256., 366.,  68., 259., 256.,
        196., 366.,  66., 261., 256., 194., 257., 365.,  65., 261., 256., 193.,
        256., 362.,  67., 258., 256., 365.,  69., 257., 195., 260., 256., 197.,
        258., 256.,  70., 259., 256., 198., 256., 365.,  68., 260., 256., 364.,
         66., 256., 196., 259., 256., 194., 258., 256., 366.,  65., 261., 256.,
        365.,  67., 193., 260., 256., 195., 256., 363.,  69., 258., 256., 364.,
         70., 257., 256., 197., 257., 198., 256., 364.,  68., 260., 256., 365.,
         66., 256., 196., 260., 256., 194., 257., 256., 366.,  65., 262., 364.,
         67., 258., 256., 193., 258., 256., 195., 365.,  70., 362.,  69., 259.,
        256., 198., 364.,  68., 256., 367.,  66., 257., 196., 256., 194., 261.,
        256., 197., 261., 256., 373.,  9

In [6]:
Train2Save = True
if Train2Save: # Task 1: For training the predictor 
    print("Training critic ...")
    criticTr.train2save(train_loader, epochs=10, fold=5)
else: # Task 2: For testing the possible test code from professor
    epoch = 50
    for i in range(epoch):
        print(f"Epoch: {i}/{epoch}")
        for x in train_loader:
            criticTr.train(x)

Training critic ...
Start training..........CV index: 0
Epoch 0/10, CV 0/5 Train Loss: 0.692735, Val Loss: 0.692026
Epoch 1/10, CV 0/5 Train Loss: 0.691386, Val Loss: 0.685758
Epoch 2/10, CV 0/5 Train Loss: 0.644364, Val Loss: 0.267816
Epoch 3/10, CV 0/5 Train Loss: 0.541842, Val Loss: 0.205733
Epoch 4/10, CV 0/5 Train Loss: 0.476684, Val Loss: 0.193053
Epoch 5/10, CV 0/5 Train Loss: 0.431254, Val Loss: 0.185262
Epoch 6/10, CV 0/5 Train Loss: 0.397910, Val Loss: 0.174309
Epoch 7/10, CV 0/5 Train Loss: 0.371574, Val Loss: 0.166195
Epoch 8/10, CV 0/5 Train Loss: 0.350224, Val Loss: 0.158351
Epoch 9/10, CV 0/5 Train Loss: 0.332300, Val Loss: 0.154476
Start training..........CV index: 1
Epoch 0/10, CV 1/5 Train Loss: 0.154241, Val Loss: 0.172470
Epoch 1/10, CV 1/5 Train Loss: 0.150781, Val Loss: 0.157418
Epoch 2/10, CV 1/5 Train Loss: 0.146905, Val Loss: 0.151367
Epoch 3/10, CV 1/5 Train Loss: 0.143718, Val Loss: 0.144010
Epoch 4/10, CV 1/5 Train Loss: 0.140530, Val Loss: 0.134630
Epoch 5/

#### Use petrained model

In [64]:
# print("Loading critic model ...")
# critic = Critic(load_trained=True)
from hw1 import LSTMCritic
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
model = LSTMCritic().to(device)
state_dict = torch.load('critick.pth').state_dict()
model.load_state_dict(state_dict)
model.eval()

LSTMCritic(
  (embedding): Embedding(382, 10)
  (lstm): LSTM(10, 128, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

In [65]:
x = test_loader.dataset[0:5][0].to(device).long()
print(x.shape)
x

torch.Size([5, 201])


tensor([[196, 259, 256,  ..., 193, 259, 256],
        [171, 256, 216,  ..., 207, 366,  62],
        [ 26, 266, 358,  ..., 256, 370,  84],
        [ 61, 258, 256,  ..., 259, 256, 363],
        [238, 276, 256,  ..., 177, 260, 256]], device='cuda:0')

In [66]:
with torch.set_grad_enabled(False):
        
    outputs = model_(x)
    outputs = torch.argmax(outputs, dim=1) # label 1 is good and label 0 is bad

print(outputs)

tensor([0, 0, 1, 0, 1], device='cuda:0')


In [67]:
import glob
from midi2seq import process_midi_seq, random_piano

dir_ = '.'
maxlen = 200

all_midis = glob.glob(f'{dir_}/maestro-v1.0.0/2004/*.midi')
good_music_midi = process_midi_seq(all_midis=all_midis, datadir=dir_, n=100, maxlen=maxlen)
good_music = torch.tensor(good_music_midi, dtype=torch.float32)


bad_music_midi = [random_piano(n=maxlen) for _ in range(len(all_midis))]
bad_music_midi = process_midi_seq(all_midis=bad_music_midi, datadir=dir_, n=100, maxlen=maxlen)
bad_music = torch.tensor(bad_music_midi, dtype=torch.float32)

all_data = torch.cat([good_music, bad_music], dim=0)

In [69]:
good_music.shape, bad_music.shape, all_data.shape

(torch.Size([406, 201]), torch.Size([110, 201]), torch.Size([516, 201]))

In [76]:
with torch.set_grad_enabled(False):
        
    outputs = torch.argmax(model_(all_data.to(device).long()), dim=1)

    outputs ^= 1 # 1 is bad and 0 is good

print(outputs)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [81]:
outputs_total[405:]

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')

In [49]:
import numpy as np
np.savetxt('good_music.txt', outputs_total.cpu().numpy())

### Download model from remote

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as get_model
from hw1 import LSTMCritic


import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
download_model = LSTMCritic().to(device)


In [ ]:
get_model.download_file_from_google_drive(file_id='1PCdNigWgslTn6SuqKxsSY7iUhNF_gfb0',
                                    dest_path='./critic.pth',
                                    unzip=True)
state_dict = torch.load('critic.pth').state_dict()
download_model.load_state_dict(state_dict)

download_model.eval()

# Composer

In [ ]:
from hw1 import Composer

In [ ]:
import torch
import glob
from midi2seq import process_midi_seq
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
all_midis = glob.glob(f'./maestro-v1.0.0/**/*.midi')

good_music_midi = process_midi_seq(all_midis=all_midis, datadir='.', n=10000, maxlen=200)

### 1. For Training

In [ ]:
cps = Composer()

In [ ]:
from hw1 import ComposerDataset

In [ ]:
sequence_length = 100
batch_size = 32

data_loader = ComposerDataset(good_music_midi, sequence_length)
data_loader = DataLoader(data_loader, batch_size, shuffle=True)

In [ ]:
cps.train2save(data_loader, epochs=10)

### 2. For Test code

In [ ]:
torch.from_numpy(good_music_midi).shape

In [ ]:
# piano_seq = torch.from_numpy(good_music_midi)
# loader = DataLoader(TensorDataset(piano_seq), shuffle=True, batch_size=32)
# epochs = 10
# cps = Composer()
# for i in range(epochs):
#     for x in loader:
#         cps.train(x[0].cuda(0).long())
            
# midi = cps.compose(100)
# midi = seq2piano(midi)
# midi.write('piano1.midi')